In [1]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.4 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=7250ac5d3bf92991cd1bd4e6fe8e21e06fbc534936f89c210207257a540cbc49
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import pickle as pkl

In [3]:
wandb_project_name = "crowd-forecasting"

scaler_save_path = '/content/drive/Othercomputers/My Laptop/github-repositories/snfc-crowd-forecasting/models/gru/scaler.pkl'
scaler_reload_path = '/content/drive/Othercomputers/My Laptop/github-repositories/snfc-crowd-forecasting/models/gru/scaler.pkl'

In [4]:
import wandb
from wandb.keras import WandbCallback

wandb.init(project=wandb_project_name)

#wandb
wandb_callback = WandbCallback(monitor='val_loss', mode='min', log_gradients=False,save_model=False)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
# Read full dataset (one hot encoding)
train = pd.read_csv('/content/drive/Othercomputers/My Laptop/github-repositories/snfc-crowd-forecasting/data/train.csv')
test = pd.read_csv('/content/drive/Othercomputers/My Laptop/github-repositories/snfc-crowd-forecasting/data/test.csv')

In [6]:
# Re-train on full data, to upload to dashboard competition benchmark
train_on_full = False

In [7]:
if(train_on_full == True):
    data = [train, test]
    train = pd.concat(data)

    train.reset_index(drop=True, inplace=True)

    test = train

In [8]:
train.columns

Index(['month', 'day', 'hour', 'train', 'station', 'p1q0', 'p2q0', 'p3q0',
       'p0q1', 'p0q2', 'p0q3', 'p0q0'],
      dtype='object')

In [14]:
from sklearn.preprocessing import OrdinalEncoder
enc_train = OrdinalEncoder()
train['train_'] = enc_train.fit_transform(train['train'].to_numpy().reshape(-1,1))

enc_station = OrdinalEncoder()
train['station_'] = enc_station.fit_transform(train['station'].to_numpy().reshape(-1,1))

test['train_'] = enc_train.transform(test['train'].to_numpy().reshape(-1,1))
test['station_'] = enc_station.transform(test['station'].to_numpy().reshape(-1,1))




In [30]:
with open('/content/drive/Othercomputers/My Laptop/github-repositories/snfc-crowd-forecasting/models/gru/scaler_train.pkl', "wb") as outfile:
            pkl.dump(enc_train, outfile)

with open('/content/drive/Othercomputers/My Laptop/github-repositories/snfc-crowd-forecasting/models/gru/scaler_station.pkl', "wb") as outfile:
            pkl.dump(enc_station, outfile)



In [15]:
# Splitting of features

x_train = train.loc[:, train.columns != 'p0q0']
y_train = train.p0q0

# Shuffle
x_train, y_train = shuffle(x_train, y_train, random_state=0)

x_test = test.loc[:, test.columns != 'p0q0']
y_test = test.p0q0

print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)


(21165, 13) (21165,)
(9952, 13) (9952,)


In [16]:
# Scaling

def scale_full_dataframe(dataframe, scaler_reload_path):

    dataframe_scaled = dataframe[["p1q0","p2q0","p3q0","p0q1","p0q2","p0q3"]]
    dataframe = dataframe.drop(columns=["p1q0","p2q0","p3q0","p0q1","p0q2","p0q3"], axis=1) # Keep only meta-data features

    if scaler_reload_path is not None: # if this is test set
        scaler = pd.read_pickle(scaler_reload_path)
    else:
        scaler = StandardScaler()

    dataframe_scaled = pd.DataFrame(scaler.fit_transform(dataframe_scaled), columns=["p1q0","p2q0","p3q0","p0q1","p0q2","p0q3"])

    dataframe = pd.concat([dataframe, dataframe_scaled], axis=1)
    
    if scaler_reload_path is None:  # if this is training set save scaler to file
        # Save scaler to file
        
        with open(scaler_save_path, "wb") as outfile:
            pkl.dump(scaler, outfile)

    return dataframe

In [17]:
x_train.head(2)

,month,day,hour,train,station,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3,train_,station_
3148,February,Monday,08:00:00,16,BC,0.085,0.287,0.383,0.000,0.000,0.000,15.0,28.0
16357,January,Tuesday,08:00:00,49,AA,0.088,0.146,0.164,0.036,0.041,0.075,48.0,0.0


In [20]:
# All Features
x_train_sub = x_train[["p1q0","p2q0","p3q0","p0q1","p0q2","p0q3","train_",'station_']]
x_test_sub = x_test[["p1q0","p2q0","p3q0","p0q1","p0q2","p0q3",'train_','station_']]


x_train_sub = scale_full_dataframe(x_train_sub, scaler_reload_path = None)  
x_test_sub = scale_full_dataframe(x_test_sub, scaler_reload_path = scaler_reload_path)  


In [21]:
x_train_sub.head(5)

,train_,station_,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3
0,0.0,3.0,-0.792793,0.414876,0.671072,-1.304219,-1.109167,-0.938706
1,0.0,3.0,-0.775468,-0.465113,-0.499507,-1.056124,-0.855648,-0.461404
2,0.0,3.0,-1.007630,-1.050524,-0.563649,-1.304219,-1.109167,-0.938706
3,0.0,3.0,-0.226826,-0.153060,-0.360535,0.074084,-0.070357,-0.111383
4,0.0,3.0,0.616350,-0.121855,-0.355189,0.487575,0.714934,0.575932


In [22]:
x_train_prev = x_train_sub[["p1q0","p2q0","p3q0"]]
x_test_prev = x_test_sub[["p1q0","p2q0","p3q0"]]

x_train_own = x_train_sub[["p0q1","p0q2","p0q3"]]
x_test_own = x_test_sub[["p0q1","p0q2","p0q3"]]

In [23]:
x_train_own = x_train_own.to_numpy().reshape(-1,3,1)
x_test_own = x_test_own.to_numpy().reshape(-1,3,1)

x_train_prev = x_train_prev.to_numpy().reshape(-1,3,1)
x_test_prev = x_test_prev.to_numpy().reshape(-1,3,1)

In [24]:
import tensorflow as tf

input_layer = tf.keras.Input(shape=(3,1)) 
input_layer_2 = tf.keras.Input(shape=(3,1))

input_layer_3 = tf.keras.Input(shape=(1,)) 
input_layer_4 = tf.keras.Input(shape=(1,)) 



x_1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(1024, activation='tanh', return_sequences=True, kernel_initializer="HeNormal"))(input_layer)
pool_1 = tf.keras.layers.GlobalAveragePooling1D()(x_1)


x_2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(1024, activation='tanh', return_sequences=True, kernel_initializer="HeNormal"))(input_layer_2)
pool_2 = tf.keras.layers.GlobalAveragePooling1D()(x_2)


combined = tf.keras.layers.concatenate([pool_1, pool_2,input_layer_3,input_layer_4])

combined = tf.keras.layers.Dense(1024, activation = 'relu')(combined)
#combined = tf.keras.layers.BatchNormalization()(combined)
#combined = tf.keras.layers.Dropout(0.2)(combined)



# Sigmoid because we want to squash the values, the crowd factor is always between 0 and 1
output_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(combined) 

model = tf.keras.Model([input_layer,input_layer_2,input_layer_3,input_layer_4], output_layer)

model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['mse', 'mae'])


# early stopping
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta = 0.000001, patience=10, mode='min', restore_best_weights=True) 

# callbacks
callbacks_list = [wandb_callback,early_stopping_callback]

In [25]:
# Model training
from sklearn.utils import shuffle
model.fit(
    [x_train_own,x_train_prev,train['train_'],train['station_']], y_train, 
    validation_split=0.2,
    batch_size = 32, 
    epochs=2000,  #1000
    shuffle = True,
    callbacks=callbacks_list,
)

Epoch 1/2000
530/530 [==============================] - 26s 21ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0536 - val_loss: 0.0027 - val_mse: 0.0027 - val_mae: 0.0382
Epoch 2/2000
530/530 [==============================] - 8s 15ms/step - loss: 0.0030 - mse: 0.0030 - mae: 0.0399 - val_loss: 0.0039 - val_mse: 0.0039 - val_mae: 0.0506
Epoch 3/2000
530/530 [==============================] - 9s 17ms/step - loss: 0.0026 - mse: 0.0026 - mae: 0.0368 - val_loss: 0.0025 - val_mse: 0.0025 - val_mae: 0.0385
Epoch 4/2000
530/530 [==============================] - 8s 16ms/step - loss: 0.0021 - mse: 0.0021 - mae: 0.0333 - val_loss: 0.0017 - val_mse: 0.0017 - val_mae: 0.0294
Epoch 5/2000
530/530 [==============================] - 9s 18ms/step - loss: 0.0017 - mse: 0.0017 - mae: 0.0306 - val_loss: 0.0016 - val_mse: 0.0016 - val_mae: 0.0288
Epoch 6/2000
530/530 [==============================] - 9s 17ms/step - loss: 0.0018 - mse: 0.0018 - mae: 0.0307 - val_loss: 0.0024 - val_mse: 0.0024 - val_mae: 0.03

In [27]:
y_pred = model.predict([x_train_own,x_train_prev,train['train_'],train['station_']])
mae_train = mean_absolute_error(y_pred, y_train)

y_pred = model.predict([x_test_own,x_test_prev,test['train_'],test['station_']])
mae_test = mean_absolute_error(y_pred, y_test)

print("mae train: ", mae_train)
print("mae test: ", mae_test)

311/311 [==============================] - 2s 7ms/step
mae train:  0.014342206787403711
mae test:  0.02091266947365186


In [28]:
model.save('/content/drive/Othercomputers/My Laptop/github-repositories/snfc-crowd-forecasting/models/gru/')

In [31]:
# Read full dataset
test_ = pd.read_csv('/content/drive/Othercomputers/My Laptop/github-repositories/snfc-crowd-forecasting/data/Xtest.csv')

In [32]:
# Adding day name column
test_['day'] = pd.to_datetime(test_['date']).dt.day_name()
test_.drop(columns=['composition','way','date'],inplace=True)
test_['hour'] = test_.hour.fillna('07:00:00')
test_['p1q0'] = test_['p1q0'].fillna(0)
test_['p2q0'] = test_['p2q0'].fillna(0)
test_['p3q0'] = test_['p3q0'].fillna(0)
test_['p0q1'] = test_['p0q1'].fillna(0)
test_['p0q2'] = test_['p0q2'].fillna(0)
test_['p0q3'] = test_['p0q3'].fillna(0)

In [33]:
test_ = test_[["day","hour","train","station","p1q0","p2q0","p3q0","p0q1","p0q2","p0q3"]]

In [34]:
test_['train_'] = enc_train.transform(test_['train'].to_numpy().reshape(-1,1))
test_['station_'] = enc_station.transform(test_['station'].to_numpy().reshape(-1,1))


In [35]:
# All Features
_sub = test_[["p1q0","p2q0","p3q0","p0q1","p0q2","p0q3","train_",'station_']]
_sub = scale_full_dataframe(_sub, scaler_reload_path = scaler_reload_path)

In [36]:
_prev = _sub[["p1q0","p2q0","p3q0"]]
_own =  _sub[["p0q1","p0q2","p0q3"]]

In [37]:
y_pred = model.predict([_own,_prev,test_['train_'],test_['station_']])

430/430 [==============================] - 4s 6ms/step


In [38]:
y_pred.shape

(13752, 1)

In [39]:
df_result = pd.DataFrame(data = y_pred, columns = ["p0q0"])
df_result.reset_index(inplace=True)
df_result = df_result.rename(columns={"index": "ind"})
df_result.ind += 1 
df_result.head(5)
df_result['ind'] = df_result['ind'].apply(lambda x: "\"" + str(x) + "\"")
df_result.to_csv("result_gru.csv",index=False)